# ML Model Example

This example provides a simply query tutorial for InfluxDB Cloud Dedicated. 

In [18]:
import influxdb_client_3 as InfluxDBClient3
import pandas as pd
import numpy as np
from keras.layers import Input, Dense, LSTM, RepeatVector, TimeDistributed, BatchNormalization, Normalization
from keras.models import Model
from keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Model, load_model
from keras.layers import Input, Dense, LSTM, RepeatVector, TimeDistributed,  Conv1D, Bidirectional
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam


import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
from huggingface_hub import push_to_hub_keras
pio.renderers.default = "vscode"
import os, datetime


In [31]:
token = ''
host = 'eu-central-1-1.aws.cloud2.influxdata.com'    
org = 'Jay-IOx'
db = 'factory'

client = InfluxDBClient3.InfluxDBClient3(token=token,
                         host=host,
                         org=org,
                         database=db) 
    
    


In [44]:
query = '''SELECT vibration FROM machine_data WHERE "machineID" = 'machine2' AND time >= now() - 5m'''
table = client.query(query=query, language="influxql")
normal_data = table.to_pandas().drop(columns=['iox::measurement']).set_index('time').dropna()
print(normal_data)
fig = px.line(normal_data, y=["vibration"], title='raw vibration')
fig.show()

                            vibration
time                                 
2023-12-11 11:23:07.858237         90
2023-12-11 11:23:07.858638         90
2023-12-11 11:23:08.356279         90
2023-12-11 11:23:08.360763         90
2023-12-11 11:23:08.864828         90
...                               ...
2023-12-11 11:28:06.332273         90
2023-12-11 11:28:06.834423         90
2023-12-11 11:28:06.835196         90
2023-12-11 11:28:07.338530         90
2023-12-11 11:28:07.339133         90

[1109 rows x 1 columns]


In [45]:
query = '''SELECT vibration FROM machine_data WHERE "machineID" = 'machine1' AND time >= now() - 5m'''
table = client.query(query=query, language="influxql")
anom_data = table.to_pandas().drop(columns=['iox::measurement']).set_index('time').dropna()
fig2 = px.line(anom_data, y=["vibration"], title='raw vibration')
fig2.show()

In [46]:
from keras.layers import Layer
import tensorflow as tf

class SequenceLayer(Layer):
    def __init__(self, timesteps, **kwargs):
        self.timesteps = timesteps
        super(SequenceLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        super(SequenceLayer, self).build(input_shape)

    def call(self, x):
        sequences = []
        for i in range(self.timesteps):
            sequences.append(x[i:-(self.timesteps-i-1)])
        return tf.stack(sequences, axis=1)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.timesteps, input_shape[1])

# Preprocess the data
scaler = StandardScaler()
df_scaled = scaler.fit_transform(normal_data)

# Build the autoencoder model
input_dim = df_scaled.shape[1]  # number of features
timesteps = 40  # you can change this based on your sequence length

inputs = Input(shape=(timesteps, input_dim))
encoded = LSTM(16, activation='relu', return_sequences=True)(inputs)
encoded = LSTM(4, activation='relu', return_sequences=False)(encoded)
decoded = RepeatVector(timesteps)(encoded)
decoded = LSTM(4, activation='relu', return_sequences=True)(decoded)
decoded = LSTM(16, activation='relu', return_sequences=True)(decoded)
decoded = TimeDistributed(Dense(input_dim))(decoded)

autoencoder = Model(inputs, decoded)

autoencoder.compile(optimizer='adam', loss='mse')

# Train the model
X_train, X_test = train_test_split(df_scaled, test_size=0.2, random_state=42)

# Directory to store logs
log_dir = os.path.join(
    "logs",
    "fit",
    datetime.datetime.now().strftime("%Y%m%d-%H%M%S"),
)

# Creating the TensorBoard callback
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

history = autoencoder.fit(X_train, X_train, epochs=100, batch_size=16, validation_data=(X_test, X_test), verbose=1, callbacks=[tensorboard_callback])


Epoch 1/100
56/56 [==============================] - 6s 28ms/step - loss: 0.9952 - val_loss: 0.9704
Epoch 2/100
56/56 [==============================] - 1s 17ms/step - loss: 0.8093 - val_loss: 0.4817
Epoch 3/100
56/56 [==============================] - 1s 17ms/step - loss: 0.4092 - val_loss: 0.3611
Epoch 4/100
56/56 [==============================] - 1s 17ms/step - loss: 0.3220 - val_loss: 0.2828
Epoch 5/100
56/56 [==============================] - 1s 17ms/step - loss: 0.2396 - val_loss: 0.1996
Epoch 6/100
56/56 [==============================] - 1s 17ms/step - loss: 0.1731 - val_loss: 0.1503
Epoch 7/100
56/56 [==============================] - 1s 17ms/step - loss: 0.1306 - val_loss: 0.1153
Epoch 8/100
56/56 [==============================] - 1s 18ms/step - loss: 0.1021 - val_loss: 0.0916
Epoch 9/100
56/56 [==============================] - 1s 17ms/step - loss: 0.0805 - val_loss: 0.0708
Epoch 10/100
56/56 [==============================] - 1s 20ms/step - loss: 0.0620 - val_loss: 0.0544

In [47]:
threshold = 80 

# Use the Autoencoder to predict on the anomalous data
scaler = StandardScaler()
normal_scaled = scaler.fit_transform(normal_data)
predictions = autoencoder.predict(normal_scaled)

# Calculate reconstruction error for each sequence
#mse = np.mean(np.power(normal_data.values[-predictions.shape[0]:] - predictions[:, -1, :], 2), axis=1)
#mse = np.mean(np.power(normal_scaled[-predictions.shape[0]:] - predictions, 2), axis=1)

# Assuming the last prediction of each sequence corresponds to the next timestamp
relevant_predictions = predictions[:, -1, :].squeeze()

# Adjust the length of normal_scaled to match relevant_predictions
# This skips the first (timesteps - 1) entries, as they don't have corresponding predictions
normal_scaled_adjusted = normal_scaled[timesteps - 1:]

# Calculate MSE for each timestamp
mse = np.power(normal_scaled_adjusted - relevant_predictions, 2).mean(axis=1)

# Scale the MSE to a percentage
min_mse = np.min(mse)
max_mse = np.max(mse)
mse_percentage = ((mse - min_mse) / (max_mse - min_mse)) * 100


# Add 'is_anomalous' column to the DataFrame
normal_data = normal_data.iloc[timesteps - 1:].copy()
normal_data['is_anomalous'] = mse_percentage > threshold
normal_data['mse_percentage'] = mse_percentage
normal_data['threshold'] = threshold



# Create a new figure
fig = go.Figure()


# Add a line for the actual load
fig.add_trace(go.Scatter(
    x=normal_data.index,
    y=normal_data["vibration"],
    mode='lines',
    name='vibration',
    line=dict(color='green'),  # specify line color here
))

# Add a line for the anomalous load
fig.add_trace(go.Scatter(
    x=normal_data[normal_data['is_anomalous']].index,
    y=normal_data[normal_data['is_anomalous']]["vibration"],
    mode='markers',
    name='Anomalous vibration',
    marker=dict(color='red', size=5),  # specify marker color and size here
))

# Show the plot
fig.show()
normal_data.to_csv('normal_data_result.csv')

35/35 [==============================] - 1s 8ms/step


In [48]:

# Use the Autoencoder to predict on the anomalous data
scaler = StandardScaler()
scaled_anom_data= scaler.fit_transform(anom_data)
predictions = autoencoder.predict(scaled_anom_data)

# Assuming the last prediction of each sequence corresponds to the next timestamp
relevant_predictions = predictions[:, -1, :].squeeze()

# Adjust the length of normal_scaled to match relevant_predictions
# This skips the first (timesteps - 1) entries, as they don't have corresponding predictions
scaled_anom_adjusted = scaled_anom_data[timesteps - 1:]

# Calculate MSE for each timestamp
mse = np.power(scaled_anom_adjusted - relevant_predictions, 2).mean(axis=1)

# Scale the MSE to a percentage
min_mse = np.min(mse)
max_mse = np.max(mse)
mse_percentage = ((mse - min_mse) / (max_mse - min_mse)) * 100

# Add 'is_anomalous' column to the DataFrame
anom_data = anom_data.iloc[timesteps - 1:].copy()
anom_data['is_anomalous'] = mse_percentage > threshold
anom_data['mse_percentage'] = mse_percentage
anom_data['threshold'] = threshold



# Create a new figure
fig = go.Figure()


# Add a line for the actual load
fig.add_trace(go.Scatter(
    x=anom_data.index,
    y=anom_data["vibration"],
    mode='lines',
    name='vibration',
    line=dict(color='green'),  # specify line color here
))

# Add a line for the anomalous load
fig.add_trace(go.Scatter(
    x=anom_data[anom_data['is_anomalous']].index,
    y=anom_data[anom_data['is_anomalous']]["vibration"],
    mode='markers',
    name='Anomalous vibration',
    marker=dict(color='red', size=5),  # specify marker color and size here
))

# Show the plot
fig.show()
normal_data.to_csv('anom_data_result.csv')

35/35 [==============================] - 0s 5ms/step


In [49]:

push_to_hub_keras(model = autoencoder,
    repo_id = "jayclifford345/vibration_autoencoder_v2",
    tags = ["anomaly-detection", "vibration", "autoencoder"],
    plot_model=True,
    log_dir=log_dir,
    token='',
    include_optimizer=True,
    config=autoencoder.get_config()
)

2023-12-11 11:34:09.597197: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,?,1]
	 [[{{node inputs}}]]
2023-12-11 11:34:09.606312: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,?,1]
	 [[{{node inputs}}]]
2023-12-11 11:34:09.786979: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,?,16]
	 [[{{node inputs}}]]
2023-12-11 11:3

INFO:tensorflow:Assets written to: /var/folders/v3/qcttlwy972z_4w3mfbxjl5y40000gn/T/tmp37ro99zd/jayclifford345/vibration_autoencoder_v2/assets


INFO:tensorflow:Assets written to: /var/folders/v3/qcttlwy972z_4w3mfbxjl5y40000gn/T/tmp37ro99zd/jayclifford345/vibration_autoencoder_v2/assets
fingerprint.pb:   0%|          | 0.00/56.0 [00:00<?, ?B/s]
















fingerprint.pb: 100%|██████████| 56.0/56.0 [00:00<00:00, 447B/s]




fingerprint.pb: 100%|██████████| 56.0/56.0 [00:00<00:00, 209B/s]
keras_metadata.pb: 100%|██████████| 26.3k/26.3k [00:00<00:00, 92.9kB/s]
events.out.tfevents.1702294100.Jays-MacBook-Pro.local.4663.14.v2: 100%|██████████| 15.7k/15.7k [00:00<00:00, 50.6kB/s]

















events.out.tfevents.1702294095.Jays-MacBook-Pro.local.4663.13.v2: 100%|██████████| 1.87M/1.87M [00:01<00:00, 1.48MB/s]
saved_model.pb: 100%|██████████| 1.09M/1.09M [00:01<00:00, 854kB/s]


Upload 5 LFS files: 100%|██████████| 5/5 [00:01<00:00,  3.25it/s]


'https://huggingface.co/jayclifford345/vibration_autoencoder_v2/tree/main/'